In [39]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

#从北京出发，经过其他城市一次且一次
class tsp(object):
    def __init__(self,city_names=None,num_vehicles=1):
        #设置城市名
        self.df = pd.read_excel('./cities.xlsx')
        self.all_city = self.df['name'].values
        self.num_vehicles = num_vehicles
        
        if city_names is not None:
            self.city_names = city_names
            self.df = self.df[self.df['name'].isin(city_names)]
        else:
            self.city_names = self.all_city
        
    def create_data_model(self):
        data = {}
        #城市之间的距离
        temp = pd.read_excel('./distance.xlsx',index_col=0)
        #按照city_names进行筛选
        temp = temp[(temp.index.isin(self.city_names))][self.city_names]
        #print(temp)
        data['distance_matrix'] = temp.values / 1000 #按照公里维度进行换算
        #print(data['distance_matrix'])
        #print(temp)
        data['num_vehicles'] = self.num_vehicles #车的数量
        data['depot'] = 0 #下标
        return data
    
    def get_solution(self,manager, routing, solution):
        #print('总行驶里程: {} 公里'.format(solution.ObjectiveValue()))
        #记录每辆车的里程
        distance_list = []
        #记录每辆车的路径规划
        route_list = []
        #遍历每辆车
        for vehicle_id in range(self.num_vehicles):
            #初始化距离为零
            route_distance = 0
            #针对第vehicle_id辆车进行规划
            route = []
            #Start后面是vehicle_id
            #从vehicle_id的起始点出发
            index = routing.Start(vehicle_id)            
            #判断这辆车的路径是否结束
            while not routing.IsEnd(index):
                #使用indextonode将manager中的index转换为distance_matrix中的index
                index_show = manager.IndexToNode(index)
                #添加到route
                route.append(index_show)
                previous_index = index
                #走到下一个节点
                index = solution.Value(routing.NextVar(index))
                #fromIndex，toIndex，vehicle id
                #针对vehicle=0，统计从previous_index到index节点的距离
                route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
            route_list.append(route)
            distance_list.append(route_distance)
        return route_list,distance_list
    
    # 添加距离约束
    def add_distance_dimension(self,routing,transit_callback_index):
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            10000,  # 车辆最大行驶距离
            True,  # start cumul to zero
            dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        # 尽量减少车辆之间的最大距离
        distance_dimension.SetGlobalSpanCostCoefficient(10)
        # SetGlobalSpanCostCoefficient是加了另一个约束条件，让总体距离尽可能小的情况下，让每辆车跑的距离和其他的车跑的距离尽量缩短
        # 此种情况应该会被用于多人分配任务的情况

    
    def work(self):
        # Step1，初始化数据，得到三个参数的字典
        data = self.create_data_model()

        # Step2，创建路线管理，tsp_size（城市数量）, num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # Step3，创建 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # Step4，注册距离，计算两点之间的距离，输入的是两个节点index，输出的是节点之间的距离
        def distance_callback(from_index, to_index):
            #将index转化为distance_matrix中的节点index
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]
        #注册距离函数
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.arc指的是边
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
        
        #添加距离维度
        self.add_distance_dimension(routing,transit_callback_index)
        
        # Step5，设置参数策略 Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        #定义最优化问题的目标
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)#路径最优策略

        # Step6，求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # Step7，输出结果
        route,route_distance = self.get_solution(manager, routing, solution)
        return route,route_distance
        
if __name__ == '__main__':
    
    #city_names = ['上海','广州','成都']
    #model = tsp(city_names=city_names)
    model = tsp(num_vehicles=5)
    #model.create_data_model()
    route,route_distance = model.work()
    print(route)
    print(route_distance)
    print(sum(route_distance))

[[0, 17, 21], [0, 5, 7, 20, 24, 23, 27, 26, 29, 28, 14], [0, 6, 19, 18, 22], [0, 15, 8, 9, 10, 32, 12, 25, 11, 31, 30, 13, 16, 1], [0, 3, 2, 4]]
[7115, 5820, 5994, 6422, 2464]
27815
